In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold,StratifiedKFold,RepeatedStratifiedKFold
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
import timeit
from xgboost import XGBClassifier

In [ ]:
dataset=pd.read_csv('train_data.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset.describe(include='object')

In [ ]:
dataset.isnull().sum()

In [ ]:
dataset['City_Code_Patient'].fillna(0,inplace=True)
dataset['Bed Grade'].fillna(0,inplace=True)

In [ ]:
dataset=dataset.drop(['case_id','patientid'],axis=1)

In [ ]:
def frequency_encoding(dataset,column):
    fe=dataset.groupby([column]).size()/len(dataset)
    dataset[column]=dataset[column].apply(lambda x: fe[x])
    return dataset

In [ ]:
le=LabelEncoder()

for col in dataset.select_dtypes(include='object').columns:
    if col not in ['Age','Stay']:
        frequency_encoding(dataset,col)
    else:
        dataset[col]=le.fit_transform(dataset[col])

In [ ]:
dataset[['Bed Grade','City_Code_Patient']]=dataset[['Bed Grade','City_Code_Patient']].astype('int')

In [ ]:
X=dataset.drop('Stay',axis=1)
y=dataset['Stay']

In [ ]:
lgbm_acc=[]

kf=KFold(n_splits=5,shuffle=True)

for i,(train_index,test_index) in enumerate(kf.split(X)):
        
    x_train, y_train = X.iloc[train_index,:], y.iloc[train_index] 
    x_test, y_test   = X.iloc[test_index,:], y.iloc[test_index]
    
    print('\nFold : {}\n'.format(i+1))
    
    lg=LGBMClassifier(learning_rate=0.04,objective='multi_class',num_class=11,n_estimators=1000,metric='multi_error') 
    
    lg.fit(x_train, y_train,eval_metric='multi_error',eval_set=[(x_train, y_train),(x_test, y_test)]
           ,early_stopping_rounds=100,verbose=50)

    acc = accuracy_score(y_test,lg.predict(x_test))*100
    lgbm_acc.append(acc)
    print("Score : ",acc)  
    
print('\nLGBM mean accuracy score: {}'.format((sum(acc_score)/5)))

In [ ]:
cat_acc=[]

kf=KFold(n_splits=5,shuffle=True)

for i,(train_index,test_index) in enumerate(kf.split(X)):
        
    x_train, y_train = X.iloc[train_index,:], y.iloc[train_index] 
    x_test, y_test   = X.iloc[test_index,:], y.iloc[test_index]
    
    print('\nFold : {}\n'.format(i+1))
    
    catboost=CatBoostClassifier(learning_rate=0.04,eval_metric='Accuracy', n_estimators=1000, verbose=500) 
    
    catboost.fit(x_train, y_train,eval_set=[(x_train, y_train),(x_test, y_test)],early_stopping_rounds=100,verbose=50)

    acc = accuracy_score(y_test,catboost.predict(x_test))*100
    cat_acc.append(acc)
    print("Score : ",cat_acc)  
    
print('\nCatboost mean accuracy score: {}'.format((sum(cat_acc)/5)))